# 生物序列Embedding提取教程

## 什么是Embedding？

**Embedding（嵌入向量）** 是将文本、序列或其他非结构化数据转换为数值向量的技术。在生物信息学中，embedding能够将DNA序列、蛋白质序列等生物序列转换为高维数值向量，这些向量能够：

1. **捕获序列的语义信息**：相似的序列会产生相似的向量
2. **支持机器学习**：数值向量可以直接用于各种机器学习算法
3. **降维表示**：将复杂的序列信息压缩为固定长度的向量
4. **计算相似性**：通过向量距离计算序列间的相似性

## 为什么要提取Embedding？

在生物信息学研究中，embedding提取具有重要价值：

- **序列分类**：识别DNA序列的功能类型（如启动子、增强子等）
- **序列相似性分析**：快速找到相似的生物序列
- **功能预测**：基于序列embedding预测蛋白质功能
- **进化分析**：研究序列的进化关系

## 本教程将演示：

1. 如何加载预训练的生物序列模型
2. 如何将DNA序列转换为embedding向量
3. 如何分析不同层的embedding特征
4. 理解embedding向量的含义和应用


## 步骤1：导入必要的库

首先，我们需要导入处理embedding所需的Python库：

- **torch**：PyTorch深度学习框架，用于模型推理
- **transformers**：Hugging Face的transformers库，提供预训练模型和分词器
  - **AutoModel**：自动加载预训练模型
  - **AutoTokenizer**：自动加载对应的分词器


In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

## 步骤2：加载预训练模型

这里我们加载一个专门用于生物序列的预训练模型：

- **model_path**：指定预训练模型的路径（这里使用的是生物序列专用模型）
- **tokenizer**：分词器，负责将DNA序列转换为模型可以理解的token
- **model**：预训练模型，设置`output_hidden_states=True`以获取所有层的隐藏状态


In [ ]:
# 指定模型路径
model_path = "/path/to/your/local/Genos-1.2B" # 替换为本地模型路径

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 加载本地下载好的Genos的权重
model = AutoModel.from_pretrained(model_path, 
                                  output_hidden_states=True,
                                  torch_dtype=torch.bfloat16, 
                                  trust_remote_code=True, 
                                  attn_implementation="flash_attention_2" # 使用flash_attention
  )
model.cuda() # 将模型加载到GPU
model.eval() # 将模型切换到推理模式

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


MixtralModel(
  (embed_tokens): Embedding(128, 1024, padding_idx=14)
  (layers): ModuleList(
    (0-11): 12 x MixtralDecoderLayer(
      (self_attn): MixtralAttention(
        (q_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (k_proj): Linear(in_features=1024, out_features=512, bias=False)
        (v_proj): Linear(in_features=1024, out_features=512, bias=False)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
      )
      (block_sparse_moe): MixtralSparseMoeBlock(
        (gate): Linear(in_features=1024, out_features=8, bias=False)
        (experts): ModuleList(
          (0-7): 8 x MixtralBlockSparseTop2MLP(
            (w1): Linear(in_features=1024, out_features=4096, bias=False)
            (w2): Linear(in_features=4096, out_features=1024, bias=False)
            (w3): Linear(in_features=1024, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
        )
      )
      (input_layernorm): MixtralRMSNorm((1024,), eps

## 步骤3：准备输入序列

我们创建一个DNA序列作为示例：

- **text**：随机生成指定长度的序列
- 这个序列包含了常见的DNA碱基（A、T、G、C）

**实际应用中**，你可以替换为任何真实的DNA序列。


In [3]:
import random

# 随机选取特定数目的碱基
bases = ['A', 'T', 'G', 'C']
seqs = random.choices(bases, k=8192)
# 生成输入的碱基序列
text = ''.join(seqs)
print(text)

GCTACGTGGTTCACATAGCCGTACCACCGTTCGGGCCATAGCGGCCGTGTATTGAGAGAATGGCCTTCATATCCGGTTTGGCCAGGATTATACTAGTTGAAGCCAGCGGCCTTCGTAGGGCGACGACTTATGTACCCATACGATGCTGATAGCTCAGGGTGACGCAATGCATAGACCCTTTGACAGTCAAATGTGCTGTTGAGGTACGACTGCCATTCGGTTTGATAGACAGTCCGCGTTGGCTGTTACATTGTATAGTGCGATACTATATATGCGTAATGAATCGGTACCGTTGTTATATGCTTATAATTGCTAGACGTGTGCCATATTTTGTGCGAAACAGGCCCGCGAATTGAATGCGTCCCCAAGACAAGGAAATAGCAATAATGGGGATGCTCCCGAAGATCCTGCAGTTTATCGTTAAATAAACTTCGTGATGTCAATACCAGCCTCGTATTTCATAAGAGAGCTCCAGACAGTTCATTCGGCGACACGGCGGAAGCAGTAGCCGCCGTATTACTTATCGAATGTTCCTATCAAGTGGGACCCGTTGCCGACTGCGGCTTGCGTGAAATAGACTTTTACCCAAAGCAACTCATAGCGAAGGCAGTGTCCACTCGCCCGTAACATCCCACCATGTTATCGTCGATGGTAGCAGCTATCCCAACGCGCCTACTCCCGCGAAAGACCCGAGAACTGGCGAGACGTGTTTGCCCGGACAGAACGCTGGAAAAATTCAGTGTATCTATAGTAGGCGCGTTACATGTCGACGCAAGCTTTTCATGAGTGAGTGCCGACATCAAGAATATTTAGATAATATGAGATCGCTCCAGCAATGGCATGAATGCTGGATCGTTCCGGCTCTCCACCAATGTCTTAGCCCCGCGTGGTCTCTCGAGGGACTGTAGATTGGACACTCCGTCTCTACATACGGTTTGGGACATTTGTTGGATCTGGAGGGAGCCCTTAAGGACAATCCAGGAAACCTTTGGCAGTCCCT

## 步骤4：序列编码

使用分词器将DNA序列转换为模型可以处理的格式：

- **tokenizer(text, return_tensors="pt")**：将文本序列转换为PyTorch张量
- **return_tensors="pt"**：返回PyTorch格式的张量
- **inputs**：包含编码后的序列信息，包括input_ids、attention_mask等

这一步将原始的DNA字符串转换为数字化的token序列，每个碱基或碱基组合对应一个唯一的token ID。


In [4]:
# 编码文本
inputs = tokenizer(text, return_tensors="pt")

# 查看编码后的token序列
print(inputs['input_ids'])

# 数据加载到GPU
inputs = {k: v.cuda() for k, v in inputs.items()}

tensor([[6, 5, 7,  ..., 7, 8, 5]])


## 步骤5：模型推理

通过预训练模型进行前向传播，获取embedding：

- **torch.no_grad()**：禁用梯度计算，因为我们只需要推理，不需要训练
- **model(\*\*inputs)**：将编码后的序列输入模型
- **outputs**：模型输出，包含最后一层的logits和所有层的隐藏状态

这一步是embedding提取的核心，模型会根据预训练的知识将序列转换为高维向量表示。


In [5]:
# 模型推理
with torch.no_grad():
    outputs = model(**inputs)

## 步骤6：提取各层Embedding

获取模型每一层的隐藏状态（embedding）：

- **outputs.hidden_states**：包含所有层的隐藏状态，是一个元组
- **hidden_states[i]**：第i层的embedding向量
- **shape**：每个embedding的形状为[batch_size, sequence_length, hidden_size]

**重要说明**：
- 不同层的embedding捕获不同层次的语义信息
- 浅层通常捕获局部特征（如碱基模式）
- 深层捕获更抽象的语义信息（如功能域、结构特征）


In [6]:
# 获取所有层的隐藏状态
hidden_states = outputs.hidden_states  # 元组，包含每一层的隐藏状态

# 遍历每一层的embedding并显示关键信息
for i, layer_embedding in enumerate(hidden_states):
    print(f"Layer {i} embedding ({layer_embedding.shape}): {layer_embedding[0, 0, :10]}")
    print("-" * 50)

Layer 0 embedding (torch.Size([1, 8192, 1024])): tensor([-0.0052, -0.0056, -0.0047, -0.0013, -0.0018, -0.0060, -0.0099, -0.0002,
        -0.0137, -0.0059], device='cuda:0', dtype=torch.bfloat16)
--------------------------------------------------
Layer 1 embedding (torch.Size([1, 8192, 1024])): tensor([-0.0801,  0.0388, -0.0127,  0.0366,  0.0718, -0.0270, -0.1182, -0.0239,
        -0.0439,  0.0203], device='cuda:0', dtype=torch.bfloat16)
--------------------------------------------------
Layer 2 embedding (torch.Size([1, 8192, 1024])): tensor([-0.1338, -0.0796,  0.1348, -0.1982,  0.2695,  0.0879, -0.1201, -0.1416,
         0.0037,  0.2324], device='cuda:0', dtype=torch.bfloat16)
--------------------------------------------------
Layer 3 embedding (torch.Size([1, 8192, 1024])): tensor([-0.2275, -0.1211,  0.2461, -0.2773,  0.3984,  0.0608, -0.3828, -0.0195,
        -0.1387,  0.3086], device='cuda:0', dtype=torch.bfloat16)
--------------------------------------------------
Layer 4 embeddin

---
# 使用Genos的包直接获取embedding


注意： 因为当前资源有限，目前API提供的的模型支持1.2B和10B，最大embedding的长度为**128k**，并且只返回最后一层的embedding

- 参数说明
    - sequence，序列，最大长度不超过128k
    - model_name, 模型名称，“Genos-1.2B”或者“Genos-10B”
    - pooling_method，池化方法，“mean”：平均池化，“max”：最大池化，“min”：最小池化，“last”：取最后一个token的embedding


In [7]:
from genos import create_client

client = create_client(token="<your_api_key>")

result = client.get_embedding(sequence=text, model_name="Genos-1.2B", pooling_method="mean")
print(result['result']['embedding'])

tensor([[-0.0116, -0.0371, -0.0466,  ..., -0.2812, -0.1299,  0.0261]])


---

### Embedding的应用场景

提取的embedding可以用于：

1. **序列相似性计算**：
   ```python
   # 计算两个序列的余弦相似度
   similarity = cosine_similarity(embedding1, embedding2)
   ```

2. **序列分类**：
   ```python
   # 使用embedding训练分类器
   classifier = train_classifier(embeddings, labels)
   ```

3. **聚类分析**：
   ```python
   # 对序列进行聚类
   clusters = kmeans_clustering(embeddings)
   ```

4. **降维可视化**：
   ```python
   # 使用t-SNE或PCA进行降维可视化
   reduced_embeddings = tsne.fit_transform(embeddings)
   ```


## 总结

通过本教程，我们学习了：

### 核心概念
- **Embedding**：将生物序列转换为数值向量的技术
- **预训练模型**：专门为生物序列训练的大型语言模型
- **分层特征**：不同层捕获不同层次的序列信息

### 技术流程
1. 加载预训练的生物序列模型
2. 将DNA序列编码为token
3. 通过模型推理获取embedding
4. 分析各层的特征表示

### 实际价值
- **加速研究**：快速分析大量生物序列
- **提高准确性**：利用预训练知识提升预测性能
- **支持下游任务**：为分类、聚类、相似性分析等提供基础

### 下一示例
1. 利用提取的embedding进行下游的人种预测任务
2. 利用提取的embedding进行下游的变异预测任务
3. RNA seq预测


**恭喜！** 你已经掌握了生物序列embedding提取的基本方法！
